Notebook to scrape emails from given gmail and put into sql database

In [1]:
import sqlite3
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import base64
import os.path
import pickle
import json
from datetime import datetime
import email
from email.utils import parsedate_to_datetime
from typing import List, Dict, Optional
import logging

In [2]:


class RSOEmailProcessor:
    def __init__(self, db_path: str = 'rso_emails.db'):
        self.db_path = db_path
        self.SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
        self.setup_logging()
        
    def setup_logging(self):
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler('rso_email_processor.log'),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger(__name__)

    def init_database(self):
        """Initialize SQLite database with necessary tables"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                
                # Create RSO emails table
                cursor.execute('''
                CREATE TABLE IF NOT EXISTS rso_emails (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    rso_listhost TEXT NOT NULL,
                    subject TEXT,
                    sender TEXT,
                    email_date TIMESTAMP,
                    content TEXT,
                    message_id TEXT UNIQUE,
                    processed_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
                ''')
                
                # Create RSOs table
                cursor.execute('''
                CREATE TABLE IF NOT EXISTS rsos (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    name TEXT NOT NULL,
                    listhost TEXT UNIQUE,
                    last_email_check TIMESTAMP
                )
                ''')
                
                conn.commit()
                self.logger.info("Database initialized successfully")
                
        except Exception as e:
            self.logger.error(f"Error initializing database: {str(e)}")
            raise

    def get_gmail_service(self):
        """Initialize Gmail API service"""
        creds = None
        if os.path.exists('token.pickle'):
            with open('token.pickle', 'rb') as token:
                creds = pickle.load(token)
        
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'credentials.json', self.SCOPES)
                creds = flow.run_local_server(port=8080)
            
            with open('token.pickle', 'wb') as token:
                pickle.dump(creds, token)

        return build('gmail', 'v1', credentials=creds)

    def extract_email_content(self, message) -> Dict:
        """Extract relevant content from email message"""
        try:
            headers = message['payload']['headers']
            subject = next((h['value'] for h in headers if h['name'] == 'Subject'), '')
            sender = next((h['value'] for h in headers if h['name'] == 'From'), '')
            message_id = next((h['value'] for h in headers if h['name'] == 'Message-ID'), '')
            date_str = next((h['value'] for h in headers if h['name'] == 'Date'), '')
            
            # Parse email date
            try:
                email_date = parsedate_to_datetime(date_str)
            except:
                email_date = datetime.now()

            # Get email body
            if 'parts' in message['payload']:
                parts = message['payload']['parts']
                body = ''
                for part in parts:
                    if part['mimeType'] == 'text/plain':
                        if 'data' in part['body']:
                            body += base64.urlsafe_b64decode(
                                part['body']['data']).decode()
            else:
                body = base64.urlsafe_b64decode(
                    message['payload']['body']['data']).decode()

            return {
                'subject': subject,
                'sender': sender,
                'content': body,
                'email_date': email_date,
                'message_id': message_id
            }
        except Exception as e:
            self.logger.error(f"Error extracting email content: {str(e)}")
            return None

    def load_rso_data(self, json_path: str):
        """Load RSO data from JSON file into SQLite database"""
        try:
            with open(json_path, 'r') as f:
                rsos = json.load(f)

            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                for rso in rsos:
                    listhost = rso.get('additional_info', {}).get('RSO Listhost')
                    if listhost:
                        cursor.execute('''
                        INSERT OR REPLACE INTO rsos (name, listhost)
                        VALUES (?, ?)
                        ''', (rso['name'], listhost))
                conn.commit()
                
            self.logger.info(f"Loaded RSO data from {json_path}")
        except Exception as e:
            self.logger.error(f"Error loading RSO data: {str(e)}")
            raise

    def process_new_emails(self, query: str = None):
        """Process new emails from Gmail and store in database"""
        try:
            service = self.get_gmail_service()
            
            # Get list of all RSO listhosts
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                cursor.execute('SELECT listhost FROM rsos')
                listhosts = {row[0] for row in cursor.fetchall()}

            # Query Gmail API
            query = query or 'newer_than:1d'  # Default to last day if no query provided
            results = service.users().messages().list(
                userId='me', q=query).execute()
            messages = results.get('messages', [])

            processed_count = 0
            for message in messages:
                msg = service.users().messages().get(
                    userId='me', id=message['id']).execute()
                
                email_data = self.extract_email_content(msg)
                if not email_data:
                    continue

                # Check if email is from a known listhost
                sender_domain = email_data['sender'].split('@')[-1]
                if sender_domain != 'lists.uchicago.edu':
                    continue

                sender_listhost = email_data['sender'].split('@')[0]
                if sender_listhost not in listhosts:
                    continue

                # Store in database
                with sqlite3.connect(self.db_path) as conn:
                    cursor = conn.cursor()
                    cursor.execute('''
                    INSERT OR IGNORE INTO rso_emails 
                    (rso_listhost, subject, sender, email_date, content, message_id)
                    VALUES (?, ?, ?, ?, ?, ?)
                    ''', (
                        sender_listhost,
                        email_data['subject'],
                        email_data['sender'],
                        email_data['email_date'],
                        email_data['content'],
                        email_data['message_id']
                    ))
                    
                    # Update last_email_check for the RSO
                    cursor.execute('''
                    UPDATE rsos 
                    SET last_email_check = CURRENT_TIMESTAMP
                    WHERE listhost = ?
                    ''', (sender_listhost,))
                    
                    conn.commit()
                    processed_count += 1

            self.logger.info(f"Processed {processed_count} new emails")
            return processed_count

        except Exception as e:
            self.logger.error(f"Error processing emails: {str(e)}")
            raise

    def get_rso_emails(self, rso_listhost: str, limit: int = 100) -> List[Dict]:
        """Retrieve emails for a specific RSO"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                cursor.execute('''
                SELECT subject, sender, email_date, content
                FROM rso_emails
                WHERE rso_listhost = ?
                ORDER BY email_date DESC
                LIMIT ?
                ''', (rso_listhost, limit))
                
                emails = [{
                    'subject': row[0],
                    'sender': row[1],
                    'date': row[2],
                    'content': row[3]
                } for row in cursor.fetchall()]
                
                return emails
        except Exception as e:
            self.logger.error(f"Error retrieving RSO emails: {str(e)}")
            return []

    def get_email_statistics(self) -> Dict:
        """Get statistics about stored emails"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                
                # Get total email count
                cursor.execute('SELECT COUNT(*) FROM rso_emails')
                total_emails = cursor.fetchone()[0]
                
                # Get email count per RSO
                cursor.execute('''
                SELECT r.name, COUNT(e.id) as email_count
                FROM rsos r
                LEFT JOIN rso_emails e ON r.listhost = e.rso_listhost
                GROUP BY r.name
                ORDER BY email_count DESC
                ''')
                rso_counts = dict(cursor.fetchall())
                
                # Get date range
                cursor.execute('''
                SELECT 
                    MIN(email_date) as earliest,
                    MAX(email_date) as latest
                FROM rso_emails
                ''')
                date_range = cursor.fetchone()
                
                return {
                    'total_emails': total_emails,
                    'rso_counts': rso_counts,
                    'date_range': {
                        'earliest': date_range[0],
                        'latest': date_range[1]
                    }
                }
        except Exception as e:
            self.logger.error(f"Error getting email statistics: {str(e)}")
            return {}